In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [4]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [6]:
out = chain.invoke("Bappy")
out

'Bappy'

In [7]:
type("Bappy")

str

In [8]:
type(out)

str

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [10]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [11]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [12]:
chain.invoke("Bappy")

'Bappyovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [13]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [14]:
chain.invoke("Bappy")

{'operation_a': 'Bappy', 'operation_b': 'Bappyovich'}

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [16]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [17]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [18]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [19]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'One curious fact about Roman Abramovich is that he is known for his extravagant and lavish lifestyle, which includes owning multiple luxury yachts, private jets, and properties around the world. He is also known for his close connections to various world leaders and celebrities, leading to speculation about his influence and power in global politics and business.'

## Let's see a more advanced use of RunnableParallel

In [20]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is dswithbappy?")

'dswithbappy is a platform that provides content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.'

In [21]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is dswithbappy?", "language": "Pirate English"})

"Arrr, dswithbappy be focusin' on providin' content on Data Science, AI, ML, DL, CV, NLP, Python programmin', etc. in English. Arrr!"